In [ ]:
def steady_state_solution_nd(eta, zeta, a_c, a_s):
    
    if eta==1 and zeta==0:   # TODO implement solution for this case
        print("Warning: no steady-state solution for resonant excitation of undamped oscillator!")
        V = 0 
        psi = 0
        u_c = 0
        u_s = 0
    else:        
        V2 = 1.0/( (1-eta**2)**2 + (2*eta*zeta)**2 )
        V = np.sqrt(V2)
        psi = np.arctan2(2*zeta*eta, 1-eta**2)
        u_c = V2*( (1-eta**2)*a_c - 2*zeta*eta*a_s )
        u_s = V2*( (1-eta**2)*a_s + 2*zeta*eta*a_c )
    return [u_c, u_s, V, psi]

def transient_solution_nd(tau_0, u_0, v_0, Uh_1, Uh_2, Vh_1, Vh_2, Up, Vp):
    uh_c1 = Uh_1(tau_0)
    uh_c2 = Uh_2(tau_0)
    vh_c1 = Vh_1(tau_0)
    vh_c2 = Vh_2(tau_0)
    up_0 = Up(tau_0)
    vp_0 = Vp(tau_0)
    # LGS Ax=b
    A = np.array([[uh_c1, uh_c2], [vh_c1, vh_c2]])
    b = np.array([u_0-up_0, v_0-vp_0])
    x = np.linalg.solve(A, b)
    C_1 = x[0]
    C_2 = x[1]
    return [C_1, C_2]

In [1]:
def interactive_tau_u_plot():
    @interact(i_zeta=widgets.IntSlider(min=0, max=250, value=50, step=5, description='$\zeta$ (1/100):'),
              i_eta = widgets.IntSlider(min=5, max=505, value=5, step=10, description='$\eta$ (1/100):'),
              i_u0=widgets.IntSlider(min=-25, max=25, value=10, step=1, description='$u_0$ (1/10):'),
              i_v0=widgets.IntSlider(min=-25, max=25, value=0, step=1, description='$v_0$:(1/10) '),
              i_as = widgets.IntSlider(min=-25, max=25, value=0, step=1, description='$a_s$ (1/10):'),
              i_ac = widgets.IntSlider(min=-25, max=25, value=0, step=1, description='$a_c$ (1/10):'),
              i_Nend = widgets.IntSlider(min=1, max=50, value=5, step=1, description='$\\tau_{end}$ (T):')
             )

    def plot_now(i_zeta, i_eta, i_u0, i_v0, i_as, i_ac, i_Nend):
        zeta = i_zeta/100.0
        eta = i_eta/100.0
        u_0 = i_u0/10.0
        v_0 = i_v0/10.0
        a_s = i_as/10.0
        a_c = i_ac/10.0
        N_end = i_Nend
        tau_0 = 0.0
        
        A = lambda tau: a_c*np.cos(eta*tau) + a_s*np.sin(eta*tau)    

        [u_c, u_s, V, psi] = steady_state_solution_nd(eta, zeta, a_c, a_s)  # nondim: omega_0=1.0
        Up = lambda tau:   u_c*np.cos(eta*tau) + u_s*np.sin(eta*tau)
        Vp = lambda tau: (-u_c*np.sin(eta*tau) + u_s*np.cos(eta*tau)) * eta
     
        if zeta < 1:
            eta_1 = np.sqrt(1-zeta**2)
            delta = zeta
            Uh_1 = lambda tau: np.exp(-delta*tau)*np.cos(eta_1*tau)
            Uh_2 = lambda tau: np.exp(-delta*tau)*np.sin(eta_1*tau)
            Vh_1 = lambda tau: np.exp(-delta*tau)*( -delta*np.cos(eta_1*tau) - eta_1*np.sin(eta_1*tau) )
            Vh_2 = lambda tau: np.exp(-delta*tau)*( -delta*np.sin(eta_1*tau) + eta_1*np.cos(eta_1*tau) )
        elif zeta > 1:
            delta_1 = zeta - np.sqrt(zeta**2 - 1)
            delta_2 = zeta + np.sqrt(zeta**2 - 1)
            Uh_1 = lambda tau: np.exp(-delta_1*tau)
            Uh_2 = lambda tau: np.exp(-delta_2*tau)
            Vh_1 = lambda tau: np.exp(-delta_1*tau)*(-delta_1)
            Vh_2 = lambda tau: np.exp(-delta_2*tau)*(-delta_2)
        else:
            delta = zeta
            Uh_1 = lambda tau: np.exp(-delta*tau)
            Uh_2 = lambda tau: tau*np.exp(-delta*tau)
            Vh_1 = lambda tau:-np.exp(-delta*tau)*delta
            Vh_2 = lambda tau: np.exp(-delta*tau) - tau*np.exp(-delta*tau)*delta     
        
        [C_1, C_2] = transient_solution_nd(tau_0, u_0, v_0, Uh_1, Uh_2, Vh_1, Vh_2, Up, Vp)
        Uh = lambda tau: C_1*Uh_1(tau) + C_2*Uh_2(tau) 
        U = lambda tau:  Uh(tau) + Up(tau)        
        
        tau_end = N_end*2*np.pi
        tau_sample = np.linspace(tau_0, tau_end , N_end*100)   # discretization with 250 points in time
        A_sample = A(tau_sample)
        Uh_sample = Uh(tau_sample)
        Up_sample = Up(tau_sample)
        U_sample = U(tau_sample)

        fig, ax1 = plt.subplots()
        color = 'tab:blue'
        ax1.set_xlabel('$\\tau$')
        ax1.set_ylabel('$u$', color=color)
        ax1.plot(tau_sample, U_sample, color=color)
        ax1.tick_params(axis='y', labelcolor=color)
        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
        color = 'tab:red'
        ax2.spines['right'].set_visible(True)
        ax2.spines['top'].set_visible(True)
        ax2.set_ylabel('$a$', color=color)  # we already handled the x-label with ax1
        ax2.plot(tau_sample, A_sample, color=color)
        ax2.tick_params(axis='y', labelcolor=color) 
        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        plt.show()
        
        #print(zeta, eta, u_0, v_0, a_s, a_c)